In [15]:
pip install python_speech_features

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip


In [16]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tempfile import TemporaryFile
import os
import math
import pickle
import random
import operator

In [17]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: c:\users\asus\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip


In [18]:
def getNeighbour(DS, pos, k):
    
    dist = []
    
    for i in range(len(DS)):
        dist1 = distance(DS[i], pos, k) + distance(pos, DS[i], k)
        dist.append((DS[i][2], dist1))

    dist.sort(key=operator.itemgetter(1))
    neighbour = []
    
    for i in range(k):
        neighbour.append(dist[i][0])
    
    return neighbour

In [19]:
def distance(inst1, inst2, k):
    dist = 0
    mm1 = inst1[0]
    cm1 = inst1[1]
    mm2 = inst2[0]
    cm2 = inst2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2 - mm1).transpose(), np.linalg.inv(cm2)), mm2 - mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [20]:
def classFinder(neighbour):
    
    sameClass = {}
    
    for i in range(len(neighbour)):
        if neighbour[i] in sameClass:
            sameClass[neighbour[i]] += 1
        else:
            sameClass[neighbour[i]] = 1
    
    sortedClass = sorted(sameClass.items(), key = operator.itemgetter(1), reverse = True)
    
    return sortedClass[0][0]

In [21]:
def accuracy(DS, predicted):
    right = 0
    
    for i in range(len(DS)):
        if DS[i][-1] == predicted[i] :
            right += 1
        
    return 1.0*right / len(DS)

In [22]:
add = r'C:\Users\Asus\OneDrive\Desktop\mini project 5th sem\genres_original'
f = open("dataset.dat", "wb")
i = 0
for folder in os.listdir(add):
    #print(folder)
    i += 1
    if i == 11:
        break
    for file in os.listdir(add + "/" + folder):
#     for file in os.listdir(os.path.join(add,folder)):
        # print(file)
        try :
            (rate, sign) = wav.read(add + "/" + folder + "/" + file)
            mfcc_features = mfcc(sign, rate, winlen = 0.010, appendEnergy = False)
            covariance = np.cov(np.matrix.transpose(mfcc_features))
            mean_matrix = mfcc_features.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print("Got an exception : ", e);
f.close()

Got an exception :  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported.


In [23]:
dataset = []

def loadDS(filename, split, trainS, testS):
    with open('dataset.dat', 'rb') as f:
        while True:
            try :
                dataset.append(pickle.load(f))
            except EOFError :
                f.close()
                break
    for i in range(len(dataset)):
        if random.random() < split :
            trainS.append(dataset[i])
        else :
            testS.append(dataset[i])
            
trainSet = []
testSet = []
loadDS('dataset.dat', 0.70, trainSet, testSet)

In [24]:
len(trainSet)

675

In [25]:
len(testSet)

324

In [26]:
predictions = []

for i in range(len(testSet)):
    predictions.append(classFinder(getNeighbour(trainSet, testSet[i], 10)))
    
acc1 = accuracy(testSet, predictions)
print(acc1*100)

67.5925925925926


In [27]:
from collections import defaultdict
results = defaultdict(int)

directory = r'C:\Users\Asus\OneDrive\Desktop\mini project 5th sem\genres_original'

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1
    
print(results)

defaultdict(<class 'int'>, {1: 'blues', 2: 'classical', 3: 'country', 4: 'disco', 5: 'hiphop', 6: 'jazz', 7: 'metal', 8: 'pop', 9: 'reggae', 10: 'rock'})


In [28]:
predict = classFinder(getNeighbour(dataset, feature, 10))
print(results[predict])

rock
